In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

In [5]:
train_dir = "./archive/tomato/train"
val_dir = "./archive/tomato/val"

In [6]:
img_size = 224
batch_size = 32

In [7]:
train_datagen = ImageDataGenerator(rescale= 1./255)
train_generator = train_datagen.flow_from_directory(train_dir, target_size = (img_size,img_size), batch_size = batch_size, class_mode = 'categorical')

Found 10000 images belonging to 10 classes.


In [8]:
val_datagen = ImageDataGenerator(rescale= 1./255)
val_generator = val_datagen.flow_from_directory(val_dir, target_size = (img_size,img_size), batch_size = batch_size, class_mode = 'categorical')

Found 984 images belonging to 10 classes.


In [9]:
list(train_generator.class_indices)

['Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Tomato___Target_Spot',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Tomato_mosaic_virus',
 'Tomato___healthy']

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

In [11]:
model = Sequential()

model.add((Conv2D(32, (3,3), activation = 'relu', input_shape = (img_size, img_size, 3))))
model.add(BatchNormalization())
model.add((MaxPooling2D(2,2)))
model.add((Conv2D(64, (3,3), activation = 'relu')))
model.add(BatchNormalization())
model.add((MaxPooling2D(2,2)))
model.add((Conv2D(64, (3,3), activation = 'relu')))
model.add(BatchNormalization())
model.add((MaxPooling2D(2,2)))
model.add((Conv2D(128, (3,3), activation = 'relu')))
model.add(BatchNormalization())
model.add((MaxPooling2D(2,2)))

model.add(Flatten())

model.add((Dense(128, activation = 'relu')))
model.add((Dropout(0.2)))
model.add((Dense(64, activation = 'relu')))
model.add((Dense(train_generator.num_classes, activation = 'softmax')))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 222, 222, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 batch_normalization_1 (Bat  (None, 109, 109, 64)      256       
 chNormalization)                                                
                                                        

In [12]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [13]:
model.fit(train_generator, epochs = 2, validation_data= val_generator)

Epoch 1/2
313/313 [==============================] - 474s 1s/step - loss: 1.3559 - accuracy: 0.5969 - val_loss: 9.4988 - val_accuracy: 0.1951
Epoch 2/2
183/313 [================>.............] - ETA: 3:05 - loss: 0.8424 - accuracy: 0.7264

UnknownError: Graph execution error:

FileNotFoundError: [Errno 2] No such file or directory: './archive/tomato/train\\Tomato___Late_blight\\e3df7441-295f-4a70-833c-4127b5b738d5___GHLB2 Leaf 8760.JPG'
Traceback (most recent call last):

  File "c:\Users\Abhi\anaconda3\lib\site-packages\tensorflow\python\ops\script_ops.py", line 268, in __call__
    ret = func(*args)

  File "c:\Users\Abhi\anaconda3\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "c:\Users\Abhi\anaconda3\lib\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "c:\Users\Abhi\anaconda3\lib\site-packages\keras\src\engine\data_adapter.py", line 917, in wrapped_generator
    for data in generator_fn():

  File "c:\Users\Abhi\anaconda3\lib\site-packages\keras\src\engine\data_adapter.py", line 1064, in generator_fn
    yield x[i]

  File "c:\Us

In [24]:
loss, accuracy = model.evaluate(val_generator)
print("Loss: ", loss)
print('Accuracy', accuracy * 100) 

32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 297ms/step - accuracy: 0.7680 - loss: 0.8586
Loss:  0.9359602928161621
Accuracy 75.80000162124634


In [27]:
import numpy as np 
import matplotlib as plt
img_path = "./archive/tomato/val/Tomato___Septoria_leaf_spot/0a25f893-1b5f-4845-baa1-f68ac03d96ac___Matt.S_CG 7863.jpg"

img = load_img(img_path, target_size = (224,224))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array,axis = 0)
img_array /= 255


In [28]:
prediction = model.predict(img_array)
class_name = ['Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Tomato___Target_Spot',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Tomato_mosaic_virus',
 'Tomato___healthy']


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step


In [29]:
predicted_class = np.argmax(prediction)

print(prediction)
print(predicted_class)
print("Class Name: ", class_name[predicted_class])

[[1.4379144e-07 1.1605932e-06 1.3313984e-04 1.0966674e-01 8.8868576e-01
  8.6827276e-06 2.2626270e-07 1.8650462e-10 1.3861944e-03 1.1796819e-04]]
4
Class Name:  Tomato___Septoria_leaf_spot
